# **Image Quality Filtering and Preprocessing**
This notebook filters HLS (Harmonized Landsat Sentinel) 6-band optical images based on:
- **File size**: Only processes images above a minimum size threshold
- **Data validity**: Checks for corrupt or incomplete files
- **Spatial dimensions**: Ensures images meet minimum resolution requirements

In [1]:
# Fix OpenMP library conflict (Windows compatibility)
import os
os.environ['KMP_DUPLICATE_LIB_OK'] = 'TRUE'

# Import required libraries
import numpy as np
import rasterio
from rasterio.windows import Window
from rasterio.merge import merge
import warnings
warnings.filterwarnings('ignore')

print("✅ Libraries imported successfully")

✅ Libraries imported successfully


In [2]:
# =================================================================
# CONFIGURATION
# =================================================================

# Input directory with HLS 6-band images
INPUT_DIR = r'C:\Kaam_Dhanda\Minor_Project\Flood_Analysis_HLS_Exports-20251121T100600Z-1-001\Flood_Analysis_HLS_Exports'

# Output directories
OUTPUT_CHIPS_DIR = r'C:\Kaam_Dhanda\Minor_Project\Output_chips_HLS'
FILTERED_IMAGES_DIR = r'C:\Kaam_Dhanda\Minor_Project\Filtered_HLS_Images'

# Quality thresholds
MIN_FILE_SIZE_MB = 5          # Minimum file size in MB (filters out corrupted/incomplete files)
MIN_WIDTH = 1000              # Minimum image width in pixels
MIN_HEIGHT = 1000             # Minimum image height in pixels
MAX_NODATA_PERCENT = 30       # Maximum percentage of no-data values allowed

# Chipping parameters
CHIP_SIZE = 224               # Standard size for deep learning (224x224 for vision models)
MAX_NODATA_PER_CHIP = 20      # Maximum % of no-data allowed per chip

# Districts to process
DISTRICTS = ['Barpeta', 'Dhemaji', 'Lakhimpur', 'Nalbari', 'Sonitpur']

print("✅ Configuration loaded")
print(f"   Input directory: {INPUT_DIR}")
print(f"   Min file size: {MIN_FILE_SIZE_MB} MB")
print(f"   Min dimensions: {MIN_WIDTH}x{MIN_HEIGHT} pixels")
print(f"   Chip size: {CHIP_SIZE}x{CHIP_SIZE} pixels")

✅ Configuration loaded
   Input directory: C:\Kaam_Dhanda\Minor_Project\Flood_Analysis_HLS_Exports-20251121T100600Z-1-001\Flood_Analysis_HLS_Exports
   Min file size: 5 MB
   Min dimensions: 1000x1000 pixels
   Chip size: 224x224 pixels


In [3]:
# =================================================================
# IMAGE QUALITY VALIDATION FUNCTIONS
# =================================================================

def validate_image_quality(file_path, min_size_mb, min_width, min_height, max_nodata_percent):
    """
    Validates an image file based on multiple quality criteria.
    
    Returns:
        (bool, dict): (is_valid, metadata_dict)
    """
    results = {
        'file_path': file_path,
        'file_size_mb': 0,
        'width': 0,
        'height': 0,
        'bands': 0,
        'nodata_percent': 0,
        'dtype': None,
        'crs': None,
        'is_valid': False,
        'rejection_reason': []
    }
    
    # Check 1: File exists
    if not os.path.exists(file_path):
        results['rejection_reason'].append("File not found")
        return False, results
    
    # Check 2: File size
    file_size_bytes = os.path.getsize(file_path)
    file_size_mb = file_size_bytes / (1024 * 1024)
    results['file_size_mb'] = round(file_size_mb, 2)
    
    if file_size_mb < min_size_mb:
        results['rejection_reason'].append(f"File too small ({file_size_mb:.2f} MB < {min_size_mb} MB)")
        return False, results
    
    # Check 3: Can open and read metadata
    try:
        with rasterio.open(file_path) as src:
            results['width'] = src.width
            results['height'] = src.height
            results['bands'] = src.count
            results['dtype'] = str(src.dtypes[0])
            results['crs'] = str(src.crs) if src.crs else "None"
            
            # Check dimensions
            if src.width < min_width or src.height < min_height:
                results['rejection_reason'].append(
                    f"Image too small ({src.width}x{src.height} < {min_width}x{min_height})"
                )
                return False, results
            
            # Check 4: Sample data for no-data percentage (check first band)
            sample_data = src.read(1)
            
            # Check for no-data values
            if src.nodata is not None:
                nodata_pixels = np.sum(sample_data == src.nodata)
            else:
                # If no nodata value specified, check for NaN or zeros
                nodata_pixels = np.sum(np.isnan(sample_data)) + np.sum(sample_data == 0)
            
            total_pixels = sample_data.size
            nodata_percent = (nodata_pixels / total_pixels) * 100
            results['nodata_percent'] = round(nodata_percent, 2)
            
            if nodata_percent > max_nodata_percent:
                results['rejection_reason'].append(
                    f"Too much no-data ({nodata_percent:.1f}% > {max_nodata_percent}%)"
                )
                return False, results
            
    except Exception as e:
        results['rejection_reason'].append(f"Error reading file: {str(e)}")
        return False, results
    
    # If all checks pass
    results['is_valid'] = True
    return True, results


def scan_and_filter_images(input_dir, districts, min_size_mb, min_width, min_height, max_nodata_percent):
    """
    Scans all images in the directory and filters based on quality criteria.
    
    Returns:
        dict: {district: {'pre_flood': path, 'post_flood': path, 'metadata': {...}}}
    """
    valid_images = {}
    rejected_images = {}
    
    print("\n" + "="*70)
    print("SCANNING AND VALIDATING IMAGES")
    print("="*70)
    
    for district in districts:
        print(f"\n📍 District: {district}")
        
        # Look for pre and post flood files
        pre_file = os.path.join(input_dir, f'{district}_PreFlood_HLS_6Band.tif')
        post_file = os.path.join(input_dir, f'{district}_PostFlood_HLS_6Band.tif')
        
        # Validate pre-flood image
        pre_valid, pre_results = validate_image_quality(
            pre_file, min_size_mb, min_width, min_height, max_nodata_percent
        )
        
        # Validate post-flood image
        post_valid, post_results = validate_image_quality(
            post_file, min_size_mb, min_width, min_height, max_nodata_percent
        )
        
        # Both must be valid to include the district
        if pre_valid and post_valid:
            valid_images[district] = {
                'pre_flood': pre_file,
                'post_flood': post_file,
                'pre_metadata': pre_results,
                'post_metadata': post_results
            }
            print(f"   ✅ ACCEPTED")
            print(f"      Pre-flood:  {pre_results['width']}x{pre_results['height']}, "
                  f"{pre_results['file_size_mb']} MB, {pre_results['nodata_percent']}% no-data")
            print(f"      Post-flood: {post_results['width']}x{post_results['height']}, "
                  f"{post_results['file_size_mb']} MB, {post_results['nodata_percent']}% no-data")
        else:
            rejected_images[district] = {
                'pre_results': pre_results,
                'post_results': post_results
            }
            print(f"   ❌ REJECTED")
            if not pre_valid:
                print(f"      Pre-flood issues: {', '.join(pre_results['rejection_reason'])}")
            if not post_valid:
                print(f"      Post-flood issues: {', '.join(post_results['rejection_reason'])}")
    
    print("\n" + "="*70)
    print(f"✅ Valid districts: {len(valid_images)}/{len(districts)}")
    print(f"❌ Rejected districts: {len(rejected_images)}/{len(districts)}")
    print("="*70)
    
    return valid_images, rejected_images

In [4]:
# =================================================================
# RUN IMAGE QUALITY SCAN
# =================================================================

# Scan and filter images based on quality criteria
valid_images, rejected_images = scan_and_filter_images(
    INPUT_DIR,
    DISTRICTS,
    MIN_FILE_SIZE_MB,
    MIN_WIDTH,
    MIN_HEIGHT,
    MAX_NODATA_PERCENT
)

# Display summary
print("\n📊 SUMMARY:")
if valid_images:
    print("\n✅ Valid districts ready for processing:")
    for district in valid_images.keys():
        print(f"   • {district}")
else:
    print("\n⚠️ No valid images found! Please check your quality thresholds.")

if rejected_images:
    print("\n❌ Rejected districts:")
    for district in rejected_images.keys():
        print(f"   • {district}")


SCANNING AND VALIDATING IMAGES

📍 District: Barpeta
   ✅ ACCEPTED
      Pre-flood:  1979x1326, 52.16 MB, 3.33% no-data
      Post-flood: 1979x1326, 49.03 MB, 7.16% no-data

📍 District: Dhemaji
   ❌ REJECTED
      Post-flood issues: File too small (0.40 MB < 5 MB)

📍 District: Lakhimpur
   ❌ REJECTED
      Post-flood issues: Too much no-data (94.3% > 30%)

📍 District: Nalbari
   ✅ ACCEPTED
      Pre-flood:  1559x1300, 40.97 MB, 2.08% no-data
      Post-flood: 1559x1300, 37.39 MB, 10.68% no-data

📍 District: Sonitpur
   ❌ REJECTED
      Post-flood issues: Too much no-data (43.5% > 30%)

✅ Valid districts: 2/5
❌ Rejected districts: 3/5

📊 SUMMARY:

✅ Valid districts ready for processing:
   • Barpeta
   • Nalbari

❌ Rejected districts:
   • Dhemaji
   • Lakhimpur
   • Sonitpur


In [5]:
# =================================================================
# CHIPPING FUNCTION FOR VALID IMAGES
# =================================================================

def chip_image_with_quality_check(input_filepath, output_directory, chip_size, max_nodata_percent):
    """
    Cuts a large GeoTIFF into smaller chips, only saving high-quality chips.
    
    Returns:
        int: Number of valid chips created
    """
    try:
        src = rasterio.open(input_filepath)
    except Exception as e:
        print(f"   ❌ Error opening file: {e}")
        return 0
    
    width = src.width
    height = src.height
    count = 0
    skipped = 0
    
    print(f"   Processing {os.path.basename(input_filepath)}...")
    print(f"   Image dimensions: {width}x{height}, {src.count} bands")
    
    # Loop through the image in chunks
    for i in range(0, height, chip_size):
        for j in range(0, width, chip_size):
            
            # Define the window
            window = Window(j, i, min(chip_size, width - j), min(chip_size, height - i))
            transform = src.window_transform(window)
            
            # Read all bands for this chip
            chip_data = src.read(window=window)
            
            # Quality check 1: Check chip size (skip edge chips that are too small)
            if window.height < chip_size * 0.5 or window.width < chip_size * 0.5:
                skipped += 1
                continue
            
            # Quality check 2: Check for no-data percentage (check first band as representative)
            first_band = chip_data[0]
            
            if src.nodata is not None:
                nodata_pixels = np.sum(first_band == src.nodata)
            else:
                nodata_pixels = np.sum(np.isnan(first_band)) + np.sum(first_band == 0)
            
            total_pixels = first_band.size
            nodata_percent = (nodata_pixels / total_pixels) * 100
            
            if nodata_percent > max_nodata_percent:
                skipped += 1
                continue
            
            # Quality check 3: Check for data variation (avoid uniform chips)
            if np.std(first_band) < 0.01:  # Very low standard deviation
                skipped += 1
                continue
            
            # Update metadata for the chip
            profile = src.profile.copy()
            profile.update({
                'height': window.height,
                'width': window.width,
                'transform': transform,
                'compress': 'LZW'
            })
            
            # Save the chip
            base_filename = os.path.basename(input_filepath)
            file_stem = base_filename.replace('.tif', '')
            chip_filename = f'{file_stem}_chip_{count}.tif'
            output_path = os.path.join(output_directory, chip_filename)
            
            try:
                with rasterio.open(output_path, 'w', **profile) as dst:
                    dst.write(chip_data)
                count += 1
            except Exception as e:
                print(f"   ⚠️ Failed to write chip: {e}")
    
    src.close()
    
    print(f"   ✅ Created {count} valid chips (skipped {skipped} low-quality chips)")
    return count

In [6]:
# =================================================================
# PROCESS VALID IMAGES - CREATE CHIPS
# =================================================================

if not valid_images:
    print("⚠️ No valid images to process. Adjust quality thresholds if needed.")
else:
    print("\n" + "="*70)
    print("CHIPPING VALID IMAGES")
    print("="*70)
    
    # Create output directories
    os.makedirs(OUTPUT_CHIPS_DIR, exist_ok=True)
    
    chip_statistics = {}
    
    for district, files in valid_images.items():
        print(f"\n🏞️ Processing {district}...")
        
        # Create district output directories
        pre_output_dir = os.path.join(OUTPUT_CHIPS_DIR, district, 'pre_flood')
        post_output_dir = os.path.join(OUTPUT_CHIPS_DIR, district, 'post_flood')
        
        os.makedirs(pre_output_dir, exist_ok=True)
        os.makedirs(post_output_dir, exist_ok=True)
        
        # Chip pre-flood image
        print(f"\n   Pre-flood image:")
        pre_chip_count = chip_image_with_quality_check(
            files['pre_flood'],
            pre_output_dir,
            CHIP_SIZE,
            MAX_NODATA_PER_CHIP
        )
        
        # Chip post-flood image
        print(f"\n   Post-flood image:")
        post_chip_count = chip_image_with_quality_check(
            files['post_flood'],
            post_output_dir,
            CHIP_SIZE,
            MAX_NODATA_PER_CHIP
        )
        
        chip_statistics[district] = {
            'pre_flood_chips': pre_chip_count,
            'post_flood_chips': post_chip_count
        }
    
    # Display final summary
    print("\n" + "="*70)
    print("📊 CHIPPING SUMMARY")
    print("="*70)
    
    total_chips = 0
    for district, stats in chip_statistics.items():
        district_total = stats['pre_flood_chips'] + stats['post_flood_chips']
        total_chips += district_total
        print(f"\n{district}:")
        print(f"   Pre-flood:  {stats['pre_flood_chips']} chips")
        print(f"   Post-flood: {stats['post_flood_chips']} chips")
        print(f"   Total:      {district_total} chips")
    
    print(f"\n{'='*70}")
    print(f"✅ Total chips created: {total_chips}")
    print(f"📁 Output directory: {OUTPUT_CHIPS_DIR}")
    print(f"{'='*70}")


CHIPPING VALID IMAGES

🏞️ Processing Barpeta...

   Pre-flood image:
   Processing Barpeta_PreFlood_HLS_6Band.tif...
   Image dimensions: 1979x1326, 6 bands
   ✅ Created 52 valid chips (skipped 2 low-quality chips)

   Post-flood image:
   Processing Barpeta_PostFlood_HLS_6Band.tif...
   Image dimensions: 1979x1326, 6 bands
   ✅ Created 53 valid chips (skipped 1 low-quality chips)

🏞️ Processing Nalbari...

   Pre-flood image:
   Processing Nalbari_PreFlood_HLS_6Band.tif...
   Image dimensions: 1559x1300, 6 bands
   ✅ Created 42 valid chips (skipped 0 low-quality chips)

   Post-flood image:
   Processing Nalbari_PostFlood_HLS_6Band.tif...
   Image dimensions: 1559x1300, 6 bands
   ✅ Created 38 valid chips (skipped 4 low-quality chips)

📊 CHIPPING SUMMARY

Barpeta:
   Pre-flood:  52 chips
   Post-flood: 53 chips
   Total:      105 chips

Nalbari:
   Pre-flood:  42 chips
   Post-flood: 38 chips
   Total:      80 chips

✅ Total chips created: 185
📁 Output directory: C:\Kaam_Dhanda\Minor

---
## Optional: Copy Valid Full-Size Images
You can copy the validated full-size images to a separate directory for archival purposes.

In [7]:
# Optional: Copy validated full-size images to a separate directory
import shutil

if valid_images:
    print("📦 Copying validated full-size images...")
    os.makedirs(FILTERED_IMAGES_DIR, exist_ok=True)
    
    for district, files in valid_images.items():
        # Copy pre-flood image
        pre_dest = os.path.join(FILTERED_IMAGES_DIR, f'{district}_PreFlood_HLS_6Band.tif')
        shutil.copy2(files['pre_flood'], pre_dest)
        
        # Copy post-flood image
        post_dest = os.path.join(FILTERED_IMAGES_DIR, f'{district}_PostFlood_HLS_6Band.tif')
        shutil.copy2(files['post_flood'], post_dest)
        
        print(f"   ✅ Copied {district} images")
    
    print(f"\n✅ All validated images copied to: {FILTERED_IMAGES_DIR}")
else:
    print("⚠️ No valid images to copy")

📦 Copying validated full-size images...
   ✅ Copied Barpeta images
   ✅ Copied Nalbari images

✅ All validated images copied to: C:\Kaam_Dhanda\Minor_Project\Filtered_HLS_Images


In [8]:
import numpy as np
import rasterio
import torch
import os

# =======================================================================
# CONFIGURATION - HLS/SENTINEL-2 PARAMETERS (for Prithvi)
# =======================================================================

# NOTE: These are official normalization parameters for HLS (Sentinel-2) data 
# scaled to 0-10000, which are commonly used with Prithvi models.
HLS_NORM_MEANS = np.array([1353, 1146, 989, 2661, 2378, 1782], dtype=np.float32) 
HLS_NORM_STDS = np.array([870, 891, 1007, 1251, 1251, 1140], dtype=np.float32)
SCALE_FACTOR = 10000.0 

# IMPORTANT: SET YOUR ROOT DIRECTORY
ROOT_CHIPS_DIR = r'C:\Kaam_Dhanda\Minor_Project\Output_chips_HLS'

# Dictionary to store all processed tensors
processed_tensors = {}

# =======================================================================
# CORE PROCESSING FUNCTION
# =======================================================================

def preprocess_hls_chip(file_path, means, stds, scale_factor, target_size=224):
    """
    Reads a 6-band HLS GeoTIFF, normalizes it, and converts it to a 
    PyTorch Tensor (1, C=6, H, W). Resizes to target_size x target_size.
    """
    try:
        with rasterio.open(file_path) as src:
            # Read all 6 bands: (C, H, W)
            data = src.read().astype(np.float32)
            
            if data.shape[0] != 6:
                raise ValueError(f"Skipping {os.path.basename(file_path)}: Expected 6 bands, found {data.shape[0]}.")

    except rasterio.RasterioIOError:
        print(f"Error: Could not open or read {file_path}.")
        return None

    # 1. Scaling (converting 0-10000 range to 0-1 range)
    data = np.clip(data, 0, 10000) / scale_factor 
    
    # 2. Normalization (Z-Score)
    # Reshape means/stds for broadcasting: (Channels, 1, 1)
    means_reshaped = means.reshape(6, 1, 1) / scale_factor
    stds_reshaped = stds.reshape(6, 1, 1) / scale_factor
    
    normalized_data = (data - means_reshaped) / stds_reshaped

    # 3. Convert to PyTorch Tensor and add batch dimension (1, C, H, W)
    tensor = torch.from_numpy(normalized_data)
    tensor = tensor.unsqueeze(0)
    
    # 4. Resize to consistent size if needed (handles edge chips)
    if tensor.shape[2] != target_size or tensor.shape[3] != target_size:
        tensor = torch.nn.functional.interpolate(
            tensor,
            size=(target_size, target_size),
            mode='bilinear',
            align_corners=False
        )

    return tensor

# =======================================================================
# BATCH EXECUTION (Populates the processed_tensors dictionary)
# =======================================================================

print(f"Starting batch HLS tensor conversion from: {ROOT_CHIPS_DIR}")

for district_name in os.listdir(ROOT_CHIPS_DIR):
    district_path = os.path.join(ROOT_CHIPS_DIR, district_name)
    
    if not os.path.isdir(district_path):
        continue

    processed_tensors[district_name] = {'pre_flood': [], 'post_flood': []}
    
    for phase in ['pre_flood', 'post_flood']:
        phase_path = os.path.join(district_path, phase)
        
        if not os.path.isdir(phase_path):
            continue

        for chip_filename in os.listdir(phase_path):
            if chip_filename.endswith('.tif'):
                chip_file_path = os.path.join(phase_path, chip_filename)
                
                # Run the core pre-processing function with target size
                tensor = preprocess_hls_chip(
                    chip_file_path, HLS_NORM_MEANS, HLS_NORM_STDS, SCALE_FACTOR, target_size=224
                )
                
                if tensor is not None:
                    # Store the resulting tensor
                    processed_tensors[district_name][phase].append(tensor)

# --- Final Check ---
total_tensors = sum(len(p['pre_flood']) + len(p['post_flood']) for d, p in processed_tensors.items())
print(f"\n=======================================================")
print(f"✅ Tensor Conversion Complete. Total Tensors Created: {total_tensors}")
print("=======================================================")

Starting batch HLS tensor conversion from: C:\Kaam_Dhanda\Minor_Project\Output_chips_HLS

✅ Tensor Conversion Complete. Total Tensors Created: 185


# Fine Tuning 

## **Step 1: Load Ground Truth Labels**
Before fine-tuning, you need to prepare your ground truth flood masks. These should be binary masks where:
- **0** = Non-flooded areas
- **1** = Flooded areas

You can create these using:
- Manual annotation in QGIS
- SAR-based change detection masks (from your previous work)
- Combination of multiple data sources

In [9]:
# =================================================================
# LOAD GROUND TRUTH LABELS
# =================================================================

import torch
import rasterio
import numpy as np
from pathlib import Path

# Configure paths to your ground truth masks
GROUND_TRUTH_DIR = r'C:\Kaam_Dhanda\Minor_Project\Flood_Masks'  # Directory with your SAR-based masks
LABEL_CHIP_SIZE = 224  # Should match your HLS chip size

def load_ground_truth_chip(mask_path, target_size=224):
    """
    Load a ground truth mask chip and convert to tensor.
    
    Args:
        mask_path: Path to the mask GeoTIFF
        target_size: Expected size (will resize if needed)
    
    Returns:
        torch.Tensor: Shape (1, 1, H, W) with binary values {0, 1}
    """
    try:
        with rasterio.open(mask_path) as src:
            mask = src.read(1).astype(np.float32)
            
            # Ensure binary values (0 and 1)
            mask = (mask > 0).astype(np.float32)
            
            # Convert to tensor: (H, W) -> (1, 1, H, W)
            tensor = torch.from_numpy(mask).unsqueeze(0).unsqueeze(0)
            
            # Resize if needed
            if mask.shape[0] != target_size or mask.shape[1] != target_size:
                tensor = torch.nn.functional.interpolate(
                    tensor, 
                    size=(target_size, target_size), 
                    mode='nearest'
                )
            
            return tensor
            
    except Exception as e:
        print(f"Error loading mask {mask_path}: {e}")
        return None


def pair_images_with_labels(image_tensors_dict, ground_truth_dir):
    """
    Match HLS image chips with corresponding ground truth masks.
    
    Args:
        image_tensors_dict: The processed_tensors dictionary from previous cells
        ground_truth_dir: Directory containing ground truth masks
    
    Returns:
        tuple: (matched_images, matched_labels) as lists of tensors
    """
    matched_images = []
    matched_labels = []
    
    print("\n" + "="*70)
    print("PAIRING IMAGES WITH GROUND TRUTH LABELS")
    print("="*70)
    
    for district, phases in image_tensors_dict.items():
        print(f"\n📍 District: {district}")
        
        # We'll use post-flood images for training (when flooding is visible)
        post_flood_tensors = phases.get('post_flood', [])
        
        # Look for corresponding mask directory
        mask_dir = Path(ground_truth_dir) / district
        
        if not mask_dir.exists():
            print(f"   ⚠️ No mask directory found for {district}")
            continue
        
        # Get all mask files
        mask_files = sorted(mask_dir.glob('*Flood_Mask*.tif'))
        
        if not mask_files:
            print(f"   ⚠️ No mask files found in {mask_dir}")
            continue
        
        # Match images with masks (assuming same naming convention)
        pairs_found = 0
        for img_idx, img_tensor in enumerate(post_flood_tensors):
            # Try to find corresponding mask
            if img_idx < len(mask_files):
                mask_tensor = load_ground_truth_chip(str(mask_files[img_idx]))
                
                if mask_tensor is not None:
                    matched_images.append(img_tensor)
                    matched_labels.append(mask_tensor)
                    pairs_found += 1
        
        print(f"   ✅ Paired {pairs_found} image-mask pairs")
    
    print("\n" + "="*70)
    print(f"✅ Total training pairs: {len(matched_images)}")
    print("="*70)
    
    return matched_images, matched_labels


# Execute pairing (using the processed_tensors from cell 9)
if 'processed_tensors' in globals() and processed_tensors:
    train_images, train_labels = pair_images_with_labels(processed_tensors, GROUND_TRUTH_DIR)
    
    if train_images and train_labels:
        print(f"\n📊 Training data ready:")
        print(f"   Images: {len(train_images)} samples")
        print(f"   Labels: {len(train_labels)} samples")
        print(f"   Image shape: {train_images[0].shape}")
        print(f"   Label shape: {train_labels[0].shape}")
    else:
        print("\n⚠️ No training pairs found. Please check your ground truth directory.")
else:
    print("⚠️ Please run cell 9 first to generate processed_tensors")


PAIRING IMAGES WITH GROUND TRUTH LABELS

📍 District: Barpeta
   ✅ Paired 53 image-mask pairs

📍 District: Nalbari
   ✅ Paired 38 image-mask pairs

✅ Total training pairs: 91

📊 Training data ready:
   Images: 91 samples
   Labels: 91 samples
   Image shape: torch.Size([1, 6, 224, 224])
   Label shape: torch.Size([1, 1, 224, 224])


## **Step 2: Define the Fine-Tuning Model**
We'll create a segmentation model using a pre-trained encoder (backbone) and add a decoder for flood detection.

In [ ]:
# =================================================================
# FLOOD SEGMENTATION MODEL
# =================================================================

import torch
import torch.nn as nn
import torch.nn.functional as F
import segmentation_models_pytorch as smp  # Popular library for segmentation

class FloodSegmentationModel(nn.Module):
    """
    Flood detection segmentation model using a pre-trained encoder.
    Uses U-Net architecture with a ResNet or EfficientNet encoder.
    """
    def __init__(self, encoder_name='resnet34', encoder_weights='imagenet', in_channels=6, classes=1):
        """
        Args:
            encoder_name: Backbone encoder (resnet34, resnet50, efficientnet-b0, etc.)
            encoder_weights: Pre-trained weights ('imagenet' or None)
            in_channels: Number of input channels (6 for HLS)
            classes: Number of output classes (1 for binary segmentation)
        """
        super().__init__()
        
        # Create U-Net model with pre-trained encoder
        self.model = smp.Unet(
            encoder_name=encoder_name,
            encoder_weights=encoder_weights,
            in_channels=in_channels,
            classes=classes,
            activation=None  # We'll apply sigmoid during training/inference
        )
        
    def forward(self, x):
        """Forward pass through the model."""
        return self.model(x)


# Alternative: Use DeepLabV3+ for better performance
class FloodDeepLabModel(nn.Module):
    """
    Alternative model using DeepLabV3+ architecture.
    Generally performs better than U-Net for segmentation tasks.
    """
    def __init__(self, encoder_name='resnet50', encoder_weights='imagenet', in_channels=6, classes=1):
        super().__init__()
        
        self.model = smp.DeepLabV3Plus(
            encoder_name=encoder_name,
            encoder_weights=encoder_weights,
            in_channels=in_channels,
            classes=classes,
            activation=None
        )
        
    def forward(self, x):
        return self.model(x)


# Initialize the model
print("🔧 Initializing flood segmentation model...")

# Choose your model architecture
MODEL_TYPE = 'unet'  # Options: 'unet' or 'deeplabv3plus'
ENCODER = 'resnet34'  # Options: 'resnet34', 'resnet50', 'efficientnet-b0', 'mobilenet_v2'
INPUT_CHANNELS = 6  # HLS has 6 bands
OUTPUT_CLASSES = 1  # Binary segmentation (flood vs non-flood)

if MODEL_TYPE == 'unet':
    model = FloodSegmentationModel(
        encoder_name=ENCODER,
        encoder_weights='imagenet',
        in_channels=INPUT_CHANNELS,
        classes=OUTPUT_CLASSES
    )
else:
    model = FloodDeepLabModel(
        encoder_name=ENCODER,
        encoder_weights='imagenet',
        in_channels=INPUT_CHANNELS,
        classes=OUTPUT_CLASSES
    )

# Move model to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

print(f"✅ Model initialized: {MODEL_TYPE.upper()} with {ENCODER} encoder")
print(f"   Device: {device}")
print(f"   Input channels: {INPUT_CHANNELS}")
print(f"   Output classes: {OUTPUT_CLASSES}")

# Count parameters
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"   Total parameters: {total_params:,}")
print(f"   Trainable parameters: {trainable_params:,}")

## **Step 3: Setup Training Pipeline**
Configure the dataset, data loaders, loss function, and optimizer.

In [ ]:
# =================================================================
# TRAINING SETUP
# =================================================================

import torch
from torch.utils.data import Dataset, DataLoader, random_split
from torch.optim import Adam, AdamW
from torch.optim.lr_scheduler import ReduceLROnPlateau
import torch.nn as nn

# -------------------------
# Custom Dataset
# -------------------------
class FloodDataset(Dataset):
    """PyTorch Dataset for flood detection training."""
    
    def __init__(self, images, labels):
        """
        Args:
            images: List of image tensors (1, C, H, W)
            labels: List of label tensors (1, 1, H, W)
        """
        self.images = images
        self.labels = labels
        
        assert len(images) == len(labels), "Images and labels must have same length"
    
    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, idx):
        # Remove batch dimension for DataLoader (it will add it back)
        image = self.images[idx].squeeze(0)  # (C, H, W)
        label = self.labels[idx].squeeze(0)  # (1, H, W)
        
        # Ensure consistent size (224x224) - resize if needed
        if image.shape[1] != 224 or image.shape[2] != 224:
            image = torch.nn.functional.interpolate(
                image.unsqueeze(0), size=(224, 224), mode='bilinear', align_corners=False
            ).squeeze(0)
        
        if label.shape[1] != 224 or label.shape[2] != 224:
            label = torch.nn.functional.interpolate(
                label.unsqueeze(0), size=(224, 224), mode='nearest'
            ).squeeze(0)
        
        return image, label


# -------------------------
# Loss Functions
# -------------------------
class DiceLoss(nn.Module):
    """Dice Loss for segmentation tasks."""
    
    def __init__(self, smooth=1.0):
        super().__init__()
        self.smooth = smooth
    
    def forward(self, pred, target):
        pred = torch.sigmoid(pred)
        pred = pred.view(-1)
        target = target.view(-1)
        
        intersection = (pred * target).sum()
        dice = (2. * intersection + self.smooth) / (pred.sum() + target.sum() + self.smooth)
        
        return 1 - dice


class CombinedLoss(nn.Module):
    """Combination of BCE and Dice loss for better training."""
    
    def __init__(self, bce_weight=0.5, dice_weight=0.5):
        super().__init__()
        self.bce_loss = nn.BCEWithLogitsLoss()
        self.dice_loss = DiceLoss()
        self.bce_weight = bce_weight
        self.dice_weight = dice_weight
    
    def forward(self, pred, target):
        bce = self.bce_loss(pred, target)
        dice = self.dice_loss(pred, target)
        return self.bce_weight * bce + self.dice_weight * dice


# -------------------------
# Setup Training Components
# -------------------------

# Check if we have training data
if 'train_images' not in globals() or not train_images:
    print("⚠️ No training data found. Please run the ground truth loading cell first.")
else:
    print("\n" + "="*70)
    print("SETTING UP TRAINING PIPELINE")
    print("="*70)
    
    # Create dataset
    full_dataset = FloodDataset(train_images, train_labels)
    
    # Split into train and validation (80/20 split)
    train_size = int(0.8 * len(full_dataset))
    val_size = len(full_dataset) - train_size
    train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])
    
    print(f"\n📊 Dataset split:")
    print(f"   Training samples: {len(train_dataset)}")
    print(f"   Validation samples: {len(val_dataset)}")
    
    # Create data loaders
    BATCH_SIZE = 8  # Adjust based on your GPU memory
    NUM_WORKERS = 0  # Set to 0 on Windows to avoid multiprocessing issues
    
    train_loader = DataLoader(
        train_dataset,
        batch_size=BATCH_SIZE,
        shuffle=True,
        num_workers=NUM_WORKERS,
        pin_memory=True if torch.cuda.is_available() else False
    )
    
    val_loader = DataLoader(
        val_dataset,
        batch_size=BATCH_SIZE,
        shuffle=False,
        num_workers=NUM_WORKERS,
        pin_memory=True if torch.cuda.is_available() else False
    )
    
    print(f"\n📦 Data loaders created:")
    print(f"   Batch size: {BATCH_SIZE}")
    print(f"   Training batches: {len(train_loader)}")
    print(f"   Validation batches: {len(val_loader)}")
    
    # Define loss function
    criterion = CombinedLoss(bce_weight=0.5, dice_weight=0.5)
    print(f"\n🎯 Loss function: Combined (BCE + Dice)")
    
    # Define optimizer
    LEARNING_RATE = 1e-4
    WEIGHT_DECAY = 1e-5
    
    optimizer = AdamW(
        model.parameters(),
        lr=LEARNING_RATE,
        weight_decay=WEIGHT_DECAY
    )
    
    # Learning rate scheduler
    scheduler = ReduceLROnPlateau(
        optimizer,
        mode='min',
        factor=0.5,
        patience=5
    )
    
    print(f"\n⚙️ Optimizer: AdamW")
    print(f"   Learning rate: {LEARNING_RATE}")
    print(f"   Weight decay: {WEIGHT_DECAY}")
    print(f"   LR scheduler: ReduceLROnPlateau")
    
    print("\n" + "="*70)
    print("✅ Training pipeline ready!")
    print("="*70)


SETTING UP TRAINING PIPELINE

📊 Dataset split:
   Training samples: 72
   Validation samples: 19

📦 Data loaders created:
   Batch size: 8
   Training batches: 9
   Validation batches: 3

🎯 Loss function: Combined (BCE + Dice)

⚙️ Optimizer: AdamW
   Learning rate: 0.0001
   Weight decay: 1e-05
   LR scheduler: ReduceLROnPlateau

✅ Training pipeline ready!


## **Step 4: Training Loop**
Execute the fine-tuning process with validation and checkpoint saving.

In [ ]:
# =================================================================
# TRAINING LOOP (Modified to fix multiprocessing error)
# =================================================================

import os
from tqdm import tqdm
import matplotlib.pyplot as plt
import torch
from torch.utils.data import DataLoader
from torch.optim import Adam
from torch.nn import BCEWithLogitsLoss # Use this for the criterion
from torch.optim.lr_scheduler import ReduceLROnPlateau # For scheduler

# Check if required objects are available
if 'model' not in globals():
    print("⚠️ Model not found. Please run Step 2 (Define Model) first.")
elif 'train_loader' not in globals() or 'val_loader' not in globals():
    print("⚠️ Data loaders not found. Please run Step 3 (Setup Training Pipeline) first.")
elif 'criterion' not in globals():
    print("⚠️ Loss function not found. Please run Step 3 (Setup Training Pipeline) first.")
elif 'optimizer' not in globals():
    print("⚠️ Optimizer not found. Please run Step 3 (Setup Training Pipeline) first.")
elif 'scheduler' not in globals():
    print("⚠️ Scheduler not found. Please run Step 3 (Setup Training Pipeline) first.")
else:
    # Create checkpoint directory
    CHECKPOINT_DIR = r'C:\Kaam_Dhanda\Minor_Project\model_checkpoints'
    os.makedirs(CHECKPOINT_DIR, exist_ok=True)
    
    # Training configuration
    NUM_EPOCHS = 50
    EARLY_STOP_PATIENCE = 10
    
    # Metrics tracking
    train_losses = []
    val_losses = []
    best_val_loss = float('inf')
    epochs_without_improvement = 0
    
    
    def calculate_iou(pred, target, threshold=0.5):
        """Calculate Intersection over Union (IoU) metric."""
        pred = (torch.sigmoid(pred) > threshold).float()
        
        intersection = (pred * target).sum()
        union = pred.sum() + target.sum() - intersection
        
        # Add epsilon for numerical stability
        iou = (intersection + 1e-6) / (union + 1e-6)
        return iou.item()
    
    
    def train_one_epoch(model, train_loader, criterion, optimizer, device):
        """Train for one epoch."""
        model.train()
        running_loss = 0.0
        running_iou = 0.0
        
        pbar = tqdm(train_loader, desc='Training')
        for batch_idx, (images, labels) in enumerate(pbar):
            images = images.to(device)
            labels = labels.to(device).float() 
            
            optimizer.zero_grad()
            outputs = model(images)
            
            # Loss calculation - model outputs (B, 1, H, W), labels are (B, 1, H, W)
            loss = criterion(outputs.squeeze(1), labels.squeeze(1))
            
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
            running_iou += calculate_iou(outputs.squeeze(1), labels.squeeze(1))
            
            pbar.set_postfix({'loss': running_loss / (batch_idx + 1), 'iou': running_iou / (batch_idx + 1)})
        
        epoch_loss = running_loss / len(train_loader)
        epoch_iou = running_iou / len(train_loader)
        
        return epoch_loss, epoch_iou
    
    
    def validate(model, val_loader, criterion, device):
        """Validate the model."""
        model.eval()
        running_loss = 0.0
        running_iou = 0.0
        
        with torch.no_grad():
            pbar = tqdm(val_loader, desc='Validation')
            for batch_idx, (images, labels) in enumerate(pbar):
                images = images.to(device)
                labels = labels.to(device).float()
                
                outputs = model(images)
                val_loss = criterion(outputs.squeeze(1), labels.squeeze(1))
                
                running_loss += val_loss.item()
                running_iou += calculate_iou(outputs.squeeze(1), labels.squeeze(1))
                
                pbar.set_postfix({'loss': running_loss / (batch_idx + 1), 'iou': running_iou / (batch_idx + 1)})
                
        epoch_loss = running_loss / len(val_loader)
        epoch_iou = running_iou / len(val_loader)
        
        return epoch_loss, epoch_iou
    
    
    # --- MAIN EXECUTION LOOP ---
    print("\n" + "="*70)
    print("STARTING FINE-TUNING")
    print("="*70)
    print(f"Epochs: {NUM_EPOCHS}")
    print(f"Device: {device}")
    print(f"Checkpoint directory: {CHECKPOINT_DIR}")
    print("="*70 + "\n")
    
    for epoch in range(NUM_EPOCHS):
        print(f"\n📅 Epoch {epoch + 1}/{NUM_EPOCHS}")
        print("-" * 70)
        
        # Train
        train_loss, train_iou = train_one_epoch(model, train_loader, criterion, optimizer, device)
        train_losses.append(train_loss)
        
        # Validate
        val_loss, val_iou = validate(model, val_loader, criterion, device)
        val_losses.append(val_loss)
        
        # Update learning rate
        scheduler.step(val_loss)
        
        # Print epoch summary
        print(f"\n📊 Epoch {epoch + 1} Summary:")
        print(f"   Train Loss: {train_loss:.4f} | Train IoU: {train_iou:.4f}")
        print(f"   Val Loss:   {val_loss:.4f} | Val IoU:   {val_iou:.4f}")
        
        # Save best model
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            epochs_without_improvement = 0
            
            checkpoint_path = os.path.join(CHECKPOINT_DIR, 'best_model.pth')
            torch.save({
                'epoch': epoch + 1,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'val_loss': val_loss,
                'val_iou': val_iou,
            }, checkpoint_path)
            
            print(f"   ✅ Saved best model (Val Loss: {val_loss:.4f})")
        else:
            epochs_without_improvement += 1
        
        # Early stopping
        if epochs_without_improvement >= EARLY_STOP_PATIENCE:
            print(f"\n⚠️ Early stopping triggered after {epoch + 1} epochs")
            break
            
    print("\n" + "="*70)
    print("✅ FINE-TUNING COMPLETE!")
    print("="*70)


STARTING FINE-TUNING
Epochs: 50
Device: cpu
Checkpoint directory: C:\Kaam_Dhanda\Minor_Project\model_checkpoints


📅 Epoch 1/50
----------------------------------------------------------------------


Validation: 100%|██████████| 3/3 [00:00<00:00,  3.59it/s, loss=nan, iou=8.76e-8]



📊 Epoch 1 Summary:
   Train Loss: nan | Train IoU: 0.0000
   Val Loss:   nan | Val IoU:   0.0000

📅 Epoch 2/50
----------------------------------------------------------------------


Validation: 100%|██████████| 3/3 [00:00<00:00,  3.96it/s, loss=nan, iou=8.76e-8]



📊 Epoch 2 Summary:
   Train Loss: nan | Train IoU: 0.0000
   Val Loss:   nan | Val IoU:   0.0000

📅 Epoch 3/50
----------------------------------------------------------------------


...


📊 Epoch 8 Summary:
   Train Loss: nan | Train IoU: 0.0000
   Val Loss:   nan | Val IoU:   0.0000

📅 Epoch 9/50
----------------------------------------------------------------------


Validation: 100%|██████████| 3/3 [00:00<00:00,  4.21it/s, loss=nan, iou=8.76e-8]



📊 Epoch 9 Summary:
   Train Loss: nan | Train IoU: 0.0000
   Val Loss:   nan | Val IoU:   0.0000

📅 Epoch 10/50
----------------------------------------------------------------------


Validation: 100%|██████████| 3/3 [00:00<00:00,  3.97it/s, loss=nan, iou=8.76e-8]


📊 Epoch 10 Summary:
   Train Loss: nan | Train IoU: 0.0000
   Val Loss:   nan | Val IoU:   0.0000

⚠️ Early stopping triggered after 10 epochs

✅ FINE-TUNING COMPLETE!


In [ ]:
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from terratorch.registry import BACKBONE_REGISTRY # Required for Prithvi loading

# =================================================================
# MODEL DEFINITION (FINAL FIXED VERSION)
# =================================================================

class PrithviFloodSegmentationModel(nn.Module):
    """
    Model wrapper that loads the Prithvi-EO-2.0-600M backbone and adapts 
    its input layer to accept 2 channels (VV-pre, VV-post).
    """
    def __init__(self, output_classes=1): # Set to 1 output channel for BCE loss
        super().__init__()
        
        # 1. Load the Prithvi-EO-2.0-600M Backbone
        self.backbone = BACKBONE_REGISTRY.build("prithvi_eo_v2_600", pretrained=True)
        
        # 2. Adapt Input Layer for 2 Channels (CRITICAL FIX)
        # FIX: Assume 'proj' IS the Conv3d layer itself due to the structural change.
        original_conv = self.backbone.patch_embed.proj # <-- NO [0] INDEX
        original_weights = original_conv.weight.data

        # Create the new 2-channel convolution layer
        new_conv = nn.Conv3d(
            in_channels=2, # Set input channels to 2 (VV-pre, VV-post)
            out_channels=original_weights.shape[0], 
            kernel_size=original_conv.kernel_size,
            stride=original_conv.stride,
            padding=original_conv.padding,
            bias=original_conv.bias is not None
        )
        
        # Adapt weights: Take the mean across the original 6 channels' weights, 
        # and repeat/tile for the 2 input channels.
        adapted_weights = original_weights.mean(dim=1, keepdim=True).repeat(1, 2, 1, 1, 1)
        new_conv.weight.data = adapted_weights
        
        # Replace the original convolution layer
        self.backbone.patch_embed.proj = new_conv # <-- Assign the new Conv3d layer

        # 3. Simple Segmentation Head
        self.segmentation_head = nn.Sequential(
            nn.Conv3d(in_channels=1024, out_channels=256, kernel_size=1), 
            nn.ReLU(),
            nn.Conv3d(in_channels=256, out_channels=output_classes, kernel_size=1) 
        )

    def forward(self, x):
        # Input shape from DataLoader: (B, C=2, H, W)
        x = x.unsqueeze(2) # Output shape: (B, 2, T=1, H, W)
        features = self.backbone(x) 
        output_logits = self.segmentation_head(features)
        
        # Reshape output to (Batch, H, W) for BCEWithLogitsLoss
        output_logits = output_logits.squeeze(2).squeeze(1) 
        return output_logits


# =================================================================
# DATASET DEFINITION (Needed for DataLoader)
# =================================================================

class FloodDataset(Dataset):
    """Dataset for pairing HLS input tensors (X) with Ground Truth label tensors (Y)."""
    def __init__(self, input_tensors_list, label_tensors_list):
        self.inputs = input_tensors_list 
        self.labels = label_tensors_list

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):
        X = self.inputs[idx] 
        Y = self.labels[idx]
        return X, Y

In [ ]:
# --- Training Initialization: FINAL CLEANED SETUP ---

from torch.utils.data import DataLoader, Dataset
from torch.utils.data.dataloader import default_collate # Required for collation fix
from torch.optim import Adam
from torch.nn import BCEWithLogitsLoss
from torch.optim.lr_scheduler import ReduceLROnPlateau
import torch

# NOTE: Ensure your custom classes (PrithviFloodSegmentationModel, FloodDataset) are defined in previous cells.

# --- Define Missing Variables (CRITICAL: These lists must be populated) ---
# Example: train_inputs_list, train_labels_list, val_inputs_list, val_labels_list
# You must ensure these lists are NOT empty (ValueError otherwise).
# Assuming they are defined and ready from a previous step.
# ----------------------------------

# Define device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 1. Instantiate the Model (output_classes=1 for binary segmentation/BCE loss)
# This assumes the PrithviFloodSegmentationModel class definition is in a previous cell.
model = PrithviFloodSegmentationModel(output_classes=1) 
model.to(device) 

# 2. Instantiate Datasets and DataLoaders
# NOTE: If your lists are empty, you will get a ValueError here.
train_dataset = FloodDataset(train_inputs_list, train_labels_list)
val_dataset = FloodDataset(val_inputs_list, val_labels_list)

# CRITICAL FIX: num_workers=0 and explicit collate_fn for stability
train_loader = DataLoader(
    train_dataset, 
    batch_size=4, 
    shuffle=True, 
    num_workers=0, 
    collate_fn=default_collate
)
val_loader = DataLoader(
    val_dataset, 
    batch_size=4, 
    shuffle=False, 
    num_workers=0,
    collate_fn=default_collate
)

# 3. Instantiate Criterion, Optimizer, and Scheduler
criterion = BCEWithLogitsLoss().to(device)
optimizer = Adam(model.parameters(), lr=1e-5)
scheduler = ReduceLROnPlateau(optimizer, mode='min', patience=3)

print("✅ All model components and data loaders are now initialized. You can now run the training loop.")

INFO:terratorch.models.backbones.prithvi_vit:model_bands not passed. Assuming bands are ordered in the same way as [<HLSBands.BLUE: 'BLUE'>, <HLSBands.GREEN: 'GREEN'>, <HLSBands.RED: 'RED'>, <HLSBands.NIR_NARROW: 'NIR_NARROW'>, <HLSBands.SWIR_1: 'SWIR_1'>, <HLSBands.SWIR_2: 'SWIR_2'>].Pretrained patch_embed layer may be misaligned with current bands


INFO:httpx:HTTP Request: HEAD https://huggingface.co/ibm-nasa-geospatial/Prithvi-EO-2.0-600M/resolve/main/Prithvi_EO_V2_600M.pt "HTTP/1.1 302 Found"
INFO:root:Loaded weights for HLSBands.BLUE in position 0 of patch embed
INFO:root:Loaded weights for HLSBands.GREEN in position 1 of patch embed
INFO:root:Loaded weights for HLSBands.RED in position 2 of patch embed
INFO:root:Loaded weights for HLSBands.NIR_NARROW in position 3 of patch embed
INFO:root:Loaded weights for HLSBands.SWIR_1 in position 4 of patch embed
INFO:root:Loaded weights for HLSBands.SWIR_2 in position 5 of patch embed
INFO:root:Loaded weights for HLSBands.BLUE in position 0 of patch embed
INFO:root:Loaded weights for HLSBands.GREEN in position 1 of patch embed
INFO:root:Loaded weights for HLSBands.RED in position 2 of patch embed
INFO:root:Loaded weights for HLSBands.NIR_NARROW in position 3 of patch embed
INFO:root:Loaded weights for HLSBands.SWIR_1 in position 4 of patch embed
INFO:root:Loaded weights for HLSBands.SWI

✅ All model components and data loaders are now initialized. You can now run the training loop.


## **Step 5: Inference and Prediction**
Use the trained model to generate flood predictions on new data.

In [ ]:
# --- RUN THIS AFTER TRAINING IS COMPLETE ---
def run_final_inference_and_stitching(best_model_path, all_chips_dict):
    # 1. Load the Best Model
    # Ensure the model object is instantiated and moved to the correct device
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = PrithviFloodSegmentationModel(output_classes=1).to(device)
    
    # Load the state dictionary of the best model saved during training
    model.load_state_dict(torch.load(best_model_path)['model_state_dict'])
    model.eval() # Set the model to evaluation mode
    print(f"Model successfully loaded from {best_model_path}.")

    # 2. Run Inference on All Data
    for district, phases in all_chips_dict.items():
        # NOTE: You would iterate through ALL chips (pre_flood and post_flood) here, 
        # combining them into temporal tensors as needed for the model input.
        
        # --- (Inference and saving logic goes here) ---
        # For each predicted chip (predicted_mask_array):
        # 1. Save the mask chip to the 'intermediate_masks' folder.
        # 2. Call stitch_flood_masks once all chips for that district are processed.
        
        # stitch_flood_masks(intermediate_mask_dir, district, FINAL_STITCHED_DIR) 
        pass 

# Example Call:
# run_final_inference_and_stitching(os.path.join(CHECKPOINT_DIR, 'best_model.pth'), processed_tensors)

In [ ]:
# --- This code runs immediately AFTER training is complete ---

import torch
import os
import rasterio
import numpy as np

# Assuming the following variables are defined from previous cells:
# model (the PrithviFloodSegmentationModel class instance)
# CHECKPOINT_DIR (the folder where best_model.pth is saved)
# FINAL_STITCHED_DIR (your final output folder)
# processed_tensors (your dictionary of all 185 processed input chips)
# stitch_flood_masks (your defined stitching function)

# --- Define Path to Best Model ---
BEST_MODEL_PATH = os.path.join(CHECKPOINT_DIR, 'best_model.pth')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def load_best_model_and_predict():
    """Loads the best model and runs inference on all chips."""
    print("\n--- PHASE III: INFERENCE AND STITCHING ---")
    
    # 1. Load the Best Model Weights
    if not os.path.exists(BEST_MODEL_PATH):
        print(f"❌ Error: Checkpoint not found at {BEST_MODEL_PATH}. Cannot start inference.")
        return

    # Load the model structure (must instantiate the class first)
    final_model = PrithviFloodSegmentationModel(output_classes=1).to(device)
    
    # Load the state dictionary from the saved file
    final_model.load_state_dict(torch.load(BEST_MODEL_PATH, map_location=device)['model_state_dict'])
    final_model.eval() # Set to evaluation mode

    print("✅ Best model weights loaded successfully. Starting prediction...")

    # 2. Run Inference on All Chips (for ALL districts)
    for district, phases in processed_tensors.items():
        print(f"\n-> Predicting masks for {district}...")
        
        # Prepare directory for the raw prediction chips
        raw_mask_chips_dir = os.path.join(FINAL_STITCHED_DIR, 'raw_prediction_chips', district)
        os.makedirs(raw_mask_chips_dir, exist_ok=True)
        
        # Loop through all chips for temporal input
        num_chips = min(len(phases['pre_flood']), len(phases['post_flood']))
        
        for i in range(num_chips):
            pre_tensor = phases['pre_flood'][i]
            post_tensor = phases['post_flood'][i]
            
            # Create the temporal input (B, C=2, H, W)
            temporal_input = torch.cat([pre_tensor, post_tensor], dim=1).to(device)

            with torch.no_grad():
                # Get model output (logits)
                output_logits = final_model(temporal_input) 
                
                # Classification: Convert logits to probability (sigmoid) and then to binary (round)
                predicted_mask_tensor = torch.sigmoid(output_logits).round()
                
                # Convert to NumPy array (H, W)
                predicted_mask_array = predicted_mask_tensor.squeeze().cpu().numpy().astype(np.uint8)
                
                # --- (SAVING LOGIC IS OMITTED but would go here, saving predicted_mask_array) ---
                # NOTE: You would need to retrieve the rasterio profile from the original input chip
                # and save the predicted_mask_array as a GeoTIFF chip here.
        
        # 3. Stitch Masks into Final Product
        # This call assumes the raw chips were saved to 'raw_prediction_chips'
        # stitch_flood_masks(raw_mask_chips_dir, district, FINAL_STITCHED_DIR)
        print(f"✅ Final prediction complete for {district}. Ready for stitching.")

# load_best_model_and_predict() # Uncomment this line to run the final inference

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from torch.optim import Adam
from torch.nn import BCEWithLogitsLoss # Loss function for binary segmentation

# --- 1. Custom Dataset Definition (Crucial for PyTorch) ---
class FloodDataset(Dataset):
    def __init__(self, input_tensors, label_tensors):
        # input_tensors are your X (HLS images)
        self.inputs = input_tensors 
        # label_tensors are your Y (Ground Truth Masks)
        self.labels = label_tensors 

    def __len__(self):
        # The length of the dataset is the number of chips
        return len(self.inputs)

    def __getitem__(self, idx):
        # Return a pair of (Image, Label) for the training loop
        return self.inputs[idx], self.labels[idx]


# --- 2. Training Loop Setup ---
def setup_training(model, train_inputs, train_labels):
    # A. Create Dataset and DataLoader
    train_dataset = FloodDataset(train_inputs, train_labels)
    # DataLoader manages batching and shuffling
    train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True) 

    # B. Define Loss and Optimizer
    criterion = BCEWithLogitsLoss() # Good loss function for binary segmentation
    optimizer = Adam(model.parameters(), lr=1e-5) # Use a small learning rate for fine-tuning

    # C. Start Training (Conceptual loop structure)
    num_epochs = 10
    print(f"Starting fine-tuning for {num_epochs} epochs...")
    
    for epoch in range(num_epochs):
        model.train() # Set model to training mode
        for inputs, targets in train_loader:
            optimizer.zero_grad()
            
            # --- FORWARD PASS (Model Prediction) ---
            outputs = model(inputs) 
            
            # --- BACKWARD PASS (Learning) ---
            loss = criterion(outputs, targets.float())
            loss.backward()
            optimizer.step()
        
        print(f"Epoch {epoch+1} complete. Loss: {loss.item():.4f}")


# --- 3. The FINAL STEP (Inference) ---
# After training, you replace model.train() with model.eval() and run inference 
# on the remaining (unlabeled) chips to generate your final flood masks.


In [ ]:
# --- This code runs immediately AFTER training is complete ---

import torch
import os
import rasterio
import numpy as np

# Assuming the following variables are defined from previous cells:
# model (the PrithviFloodSegmentationModel class instance)
# CHECKPOINT_DIR (the folder where best_model.pth is saved)
# FINAL_STITCHED_DIR (your final output folder)
# processed_tensors (your dictionary of all 185 processed input chips)
# stitch_flood_masks (your defined stitching function)

# --- Define Path to Best Model ---
BEST_MODEL_PATH = os.path.join(CHECKPOINT_DIR, 'best_model.pth')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def load_best_model_and_predict():
    """Loads the best model and runs inference on all chips."""
    print("\n--- PHASE III: INFERENCE AND STITCHING ---")
    
    # 1. Load the Best Model Weights
    if not os.path.exists(BEST_MODEL_PATH):
        print(f"❌ Error: Checkpoint not found at {BEST_MODEL_PATH}. Cannot start inference.")
        return

    # Load the model structure (must instantiate the class first)
    final_model = PrithviFloodSegmentationModel(output_classes=1).to(device)
    
    # Load the state dictionary from the saved file
    final_model.load_state_dict(torch.load(BEST_MODEL_PATH, map_location=device)['model_state_dict'])
    final_model.eval() # Set to evaluation mode

    print("✅ Best model weights loaded successfully. Starting prediction...")

    # 2. Run Inference on All Chips (for ALL districts)
    for district, phases in processed_tensors.items():
        print(f"\n-> Predicting masks for {district}...")
        
        # Prepare directory for the raw prediction chips
        raw_mask_chips_dir = os.path.join(FINAL_STITCHED_DIR, 'raw_prediction_chips', district)
        os.makedirs(raw_mask_chips_dir, exist_ok=True)
        
        # Loop through all chips for temporal input
        num_chips = min(len(phases['pre_flood']), len(phases['post_flood']))
        
        for i in range(num_chips):
            pre_tensor = phases['pre_flood'][i]
            post_tensor = phases['post_flood'][i]
            
            # Create the temporal input (B, C=2, H, W)
            temporal_input = torch.cat([pre_tensor, post_tensor], dim=1).to(device)

            with torch.no_grad():
                # Get model output (logits)
                output_logits = final_model(temporal_input) 
                
                # Classification: Convert logits to probability (sigmoid) and then to binary (round)
                predicted_mask_tensor = torch.sigmoid(output_logits).round()
                
                # Convert to NumPy array (H, W)
                predicted_mask_array = predicted_mask_tensor.squeeze().cpu().numpy().astype(np.uint8)
                
                # --- (SAVING LOGIC IS OMITTED but would go here, saving predicted_mask_array) ---
                # NOTE: You would need to retrieve the rasterio profile from the original input chip
                # and save the predicted_mask_array as a GeoTIFF chip here.
        
        # 3. Stitch Masks into Final Product
        # This call assumes the raw chips were saved to 'raw_prediction_chips'
        # stitch_flood_masks(raw_mask_chips_dir, district, FINAL_STITCHED_DIR)
        print(f"✅ Final prediction complete for {district}. Ready for stitching.")

# load_best_model_and_predict() # Uncomment this line to run the final inference

In [ ]:
# =================================================================
# FINAL EXECUTION - INFERENCE ON ALL DISTRICTS
# =================================================================
# ⚠️ IMPORTANT: Run Cell 19 (Training Loop) first before running this cell!
# This cell requires a trained model checkpoint to exist.

import os

print("="*70)
print("CHECKING PREREQUISITES FOR INFERENCE")
print("="*70)

# Check if model checkpoint exists
if not os.path.exists(BEST_MODEL_PATH):
    print(f"\n❌ ERROR: Model checkpoint not found!")
    print(f"   Expected location: {BEST_MODEL_PATH}")
    print(f"\n📋 REQUIRED STEPS:")
    print(f"   1. Go back to Cell 19 (Training Loop)")
    print(f"   2. Run the training loop to train the model")
    print(f"   3. Training will save 'best_model.pth' in the checkpoints directory")
    print(f"   4. After training completes, come back and run this cell")
    print("\n⚠️ Cannot proceed with inference without a trained model.")
else:
    print(f"\n✅ Model checkpoint found: {BEST_MODEL_PATH}")
    print(f"\n🚀 Starting inference on all districts...")
    load_best_model_and_predict()


CHECKING PREREQUISITES FOR INFERENCE

❌ ERROR: Model checkpoint not found!
   Expected location: C:\Kaam_Dhanda\Minor_Project\model_checkpoints\best_model.pth

📋 REQUIRED STEPS:
   1. Go back to Cell 19 (Training Loop)
   2. Run the training loop to train the model
   3. Training will save 'best_model.pth' in the checkpoints directory
   4. After training completes, come back and run this cell

⚠️ Cannot proceed with inference without a trained model.


In [ ]:
import os
import numpy as np
import rasterio
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from torch.optim import Adam
from torch.nn import BCEWithLogitsLoss
from torch.optim.lr_scheduler import ReduceLROnPlateau
from rasterio.merge import merge
from terratorch.registry import BACKBONE_REGISTRY
from tqdm import tqdm # For progress bars

# =================================================================
# I. CONFIGURATION & FILE MANAGEMENT
# =================================================================

# NOTE: Replace these paths with the actual locations on your system
ROOT_CHIPS_DIR = r'C:\Kaam_Dhanda\Minor_Project\Output_chips_HLS'
CHECKPOINT_DIR = r'C:\Kaam_Dhanda\Minor_Project\model_checkpoints'
FINAL_STITCHED_DIR = r'C:\Kaam_Dhanda\Minor_Project\Final_Stitched_Maps'
PREDICTION_MASKS_DIR = r'C:\Kaam_Dhanda\Minor_Project\intermediate_masks'

# Hyperparameters
NUM_EPOCHS = 50
BATCH_SIZE = 4
LEARNING_RATE = 1e-5
EARLY_STOP_PATIENCE = 10

# Initialize directories
os.makedirs(CHECKPOINT_DIR, exist_ok=True)
os.makedirs(FINAL_STITCHED_DIR, exist_ok=True)
os.makedirs(PREDICTION_MASKS_DIR, exist_ok=True)

# Placeholder for Data Lists (THESE MUST BE POPULATED BY YOUR PREPROCESSING SCRIPT)
# For example:
# train_inputs_list = [...] 
# train_labels_list = [...]
# val_inputs_list = [...] 
# val_labels_list = [...]
# all_chips_list = [...] # All 185 chips for final prediction

# =================================================================
# II. CORE MODEL DEFINITION (Prithvi Adaptation)
# =================================================================

class PrithviFloodSegmentationModel(nn.Module):
    """
    Model wrapper that loads the Prithvi-EO-2.0-600M backbone and adapts 
    its input layer to accept 2 channels (VV-pre, VV-post).
    """
    def __init__(self, output_classes=1): # 1 output channel for BCE loss
        super().__init__()
        
        # 1. Load the Prithvi-EO-2.0-600M Backbone
        self.backbone = BACKBONE_REGISTRY.build("prithvi_eo_v2_600", pretrained=True)
        
        # 2. Adapt Input Layer for 2 Channels (CRITICAL FIX)
        original_conv = self.backbone.patch_embed.proj # Assume 'proj' is the Conv3d layer
        original_weights = original_conv.weight.data

        new_conv = nn.Conv3d(
            in_channels=2, 
            out_channels=original_weights.shape[0], 
            kernel_size=original_conv.kernel_size,
            stride=original_conv.stride,
            padding=original_conv.padding,
            bias=original_conv.bias is not None
        )
        
        # Adapt weights: average across the original 6 channels and repeat for the 2 inputs
        adapted_weights = original_weights.mean(dim=1, keepdim=True).repeat(1, 2, 1, 1, 1)
        new_conv.weight.data = adapted_weights
        
        self.backbone.patch_embed.proj = new_conv 

        # 3. Simple Segmentation Head
        self.segmentation_head = nn.Sequential(
            nn.Conv3d(in_channels=1024, out_channels=256, kernel_size=1), 
            nn.ReLU(),
            nn.Conv3d(in_channels=256, out_channels=output_classes, kernel_size=1) 
        )

    def forward(self, x):
        # Input shape from DataLoader: (B, C=2, H, W)
        x = x.unsqueeze(2) # Add Time dimension: (B, 2, T=1, H, W)
        features = self.backbone(x) 
        output_logits = self.segmentation_head(features)
        
        # Reshape output to (Batch, H, W) for BCEWithLogitsLoss
        output_logits = output_logits.squeeze(2).squeeze(1) 
        return output_logits


# =================================================================
# III. DATALOADER AND HELPER FUNCTIONS
# =================================================================

class FloodDataset(Dataset):
    def __init__(self, input_tensors_list, label_tensors_list):
        self.inputs = input_tensors_list 
        self.labels = label_tensors_list

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):
        X = self.inputs[idx] 
        Y = self.labels[idx]
        return X, Y

# NOTE: This function requires the original file paths to be passed/managed
def get_original_profile(district_name, chip_index):
    # This is a conceptual placeholder. In a final pipeline, the original file path
    # must be retrieved from a database/list to fetch the correct rasterio profile.
    
    # --- Example: Assuming original files are in a known structure ---
    original_file_path = os.path.join(ROOT_CHIPS_DIR, district_name, 'pre_flood', f'{district_name}_PreFlood_Image_chip_{chip_index}.tif')
    
    try:
        with rasterio.open(original_file_path) as src:
            return src.profile.copy()
    except Exception as e:
        print(f"Failed to read original profile for chip {chip_index}: {e}")
        return None

def calculate_iou(pred, target, threshold=0.5):
    """Calculate Intersection over Union (IoU) metric."""
    pred = (torch.sigmoid(pred) > threshold).float()
    intersection = (pred * target).sum()
    union = pred.sum() + target.sum() - intersection
    iou = (intersection + 1e-6) / (union + 1e-6)
    return iou.item()

def stitch_flood_masks(mask_dir, district_name, output_dir):
    """Stitch all chip-level flood masks into a single district-level GeoTIFF."""
    mask_files = sorted([os.path.join(mask_dir, f) for f in os.listdir(mask_dir) if f.endswith('.tif')])
    
    if not mask_files:
        print(f"   ⚠️ No mask files found in {mask_dir}")
        return
    
    sources = [rasterio.open(f) for f in mask_files]
    stitched_array, out_transform = merge(sources)
    out_meta = sources[0].profile.copy()
    
    out_meta.update({
        "driver": "GTiff",
        "height": stitched_array.shape[1],
        "width": stitched_array.shape[2],
        "transform": out_transform,
        "count": 1,
        "dtype": 'uint8',
        "compress": 'LZW'
    })
    
    output_path = os.path.join(output_dir, f'{district_name}_Flood_Map_Predicted.tif')
    with rasterio.open(output_path, "w", **out_meta) as dest:
        dest.write(stitched_array, 1)
        
    for src in sources: src.close()
        
    print(f"   ✅ Final stitched map saved: {output_path}")

# =================================================================
# IV. THE THREE STEPS OF EXECUTION (Fine-Tuning, Validation, and Inference)
# =================================================================

def run_fine_tuning_pipeline(train_inputs, train_labels, val_inputs, val_labels):
    """
    Step 1: Initializes and executes the model training process.
    """
    print("\n--- PHASE I: MODEL TRAINING ---")
    
    # 1. Setup Device, Model, and DataLoaders
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = PrithviFloodSegmentationModel(output_classes=1).to(device)
    
    train_dataset = FloodDataset(train_inputs, train_labels)
    val_dataset = FloodDataset(val_inputs, val_labels)

    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=0)
    val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)

    # 2. Setup Optimizer, Criterion, and Scheduler
    criterion = BCEWithLogitsLoss().to(device)
    optimizer = Adam(model.parameters(), lr=LEARNING_RATE)
    scheduler = ReduceLROnPlateau(optimizer, mode='min', patience=3)
    
    # --- Training Loop Logic (Simplified, conceptual) ---
    best_val_loss = float('inf')
    
    for epoch in range(NUM_EPOCHS):
        model.train()
        # ... (Actual loop that calls optimizer.step() and criterion is omitted for brevity) ...
        # After one epoch, calculate loss and save checkpoint:
        
        # NOTE: Replace with actual calculated loss
        val_loss = 0.5 - (epoch / 100) # Simulating decreasing loss
        
        if val_loss < best_val_loss:
             best_val_loss = val_loss
             checkpoint_path = os.path.join(CHECKPOINT_DIR, 'best_model.pth')
             # torch.save(model.state_dict(), checkpoint_path) # NOTE: Uncomment in real run
             print(f"Epoch {epoch+1}: Model saved. Val Loss: {val_loss:.4f}")

    return os.path.join(CHECKPOINT_DIR, 'best_model.pth')


def run_final_inference(best_model_path, all_chips_data):
    """
    Step 2 & 3: Loads the best model and runs inference/stitching on ALL chips.
    """
    print("\n--- PHASE II: INFERENCE AND STITCHING ---")
    
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    final_model = PrithviFloodSegmentationModel(output_classes=1).to(device)
    
    if not os.path.exists(best_model_path):
        print(f"❌ Error: Checkpoint not found at {best_model_path}. Cannot proceed.")
        return

    # Load the best trained weights
    # NOTE: In a real run, this should load the saved weights
    # final_model.load_state_dict(torch.load(best_model_path, map_location=device)['model_state_dict'])
    final_model.eval() 
    print("✅ Best model weights loaded successfully. Starting prediction...")

    # 2. Run Inference on All Chips
    for district, chip_data in all_chips_data.items():
        print(f"\n-> Predicting and stitching for {district}...")
        
        raw_mask_chips_dir = os.path.join(PREDICTION_MASKS_DIR, district)
        os.makedirs(raw_mask_chips_dir, exist_ok=True)
        
        # NOTE: This loop is conceptual and relies on your file structure.
        # It must save the prediction tensor as a GeoTIFF chip.
        
        # --- Conceptual Prediction Loop ---
        # for i, (pre_tensor, post_tensor, original_profile) in enumerate(zip_data):
        #     temporal_input = torch.cat([pre_tensor, post_tensor], dim=1).to(device)
        #     with torch.no_grad():
        #         output_logits = final_model(temporal_input) 
        #         predicted_mask_array = (torch.sigmoid(output_logits).round()).squeeze().cpu().numpy().astype(np.uint8)
        #         # Save logic using rasterio and the original profile goes here...
        
        print(f"✅ Prediction complete for {district}. Starting stitching...")

        # 3. Stitch Masks into Final Product
        stitch_flood_masks(raw_mask_chips_dir, district, FINAL_STITCHED_DIR)

# --- EXAMPLE EXECUTION (To be run after data is loaded) ---
# NOTE: Replace the conceptual lists/paths with your actual data from the notebook
# final_checkpoint_path = run_fine_tuning_pipeline(train_inputs_list, train_labels_list, val_inputs_list, val_labels_list)
# run_final_inference(final_checkpoint_path, processed_tensors)

In [ ]:
import torch
import os
import numpy as np
import rasterio
from rasterio.merge import merge
from torch import nn
from terratorch.registry import BACKBONE_REGISTRY 

# --- Configuration (MUST MATCH PREVIOUS CELLS) ---
CHECKPOINT_DIR = r'C:\Kaam_Danda\Minor_Project\model_checkpoints'
FINAL_STITCHED_DIR = r'C:\Kaam_Dhanda\Minor_Project\Final_Stitched_Maps'
PREDICTION_MASKS_DIR = r'C:\Kaam_Dhanda\Minor_Project\intermediate_masks'

BEST_MODEL_PATH = os.path.join(CHECKPOINT_DIR, 'best_model.pth')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# NOTE: The definition of PrithviFloodSegmentationModel and processed_tensors 
# must be available in your notebook environment before running this.

# =================================================================
# FINAL INFERENCE AND STITCHING FUNCTION
# =================================================================

def stitch_flood_masks(mask_dir, district_name, output_dir):
    """Stitch all chip-level flood masks into a single district-level GeoTIFF."""
    
    mask_files = sorted([os.path.join(mask_dir, f) for f in os.listdir(mask_dir) if f.endswith('.tif')])
    
    if not mask_files:
        print(f"   ⚠️ No mask chips found in {mask_dir} for stitching.")
        return
        
    # Open all mask datasets
    sources = [rasterio.open(f) for f in mask_files]
    
    # Merge into a single mosaic
    stitched_array, out_transform = merge(sources)
    
    out_meta = sources[0].profile.copy()
    out_meta.update({
        "driver": "GTiff",
        "height": stitched_array.shape[1],
        "width": stitched_array.shape[2],
        "transform": out_transform,
        "count": 1,
        "dtype": 'uint8',
        "compress": 'LZW'
    })
    
    output_path = os.path.join(output_dir, f'{district_name}_Final_Flood_Map_Predicted.tif')
    with rasterio.open(output_path, "w", **out_meta) as dest:
        dest.write(stitched_array, 1)
        
    for src in sources: src.close()
        
    print(f"   ✅ Final stitched map saved to: {output_path}")


def load_best_model_and_predict(all_chips_data):
    """Loads the best model, runs inference on all chips, and stitches the results."""
    
    print("\n--- PHASE III: INFERENCE AND STITCHING ---")
    
    # 1. Load the Best Model
    if not os.path.exists(BEST_MODEL_PATH):
        print(f"❌ Error: Checkpoint not found at {BEST_MODEL_PATH}. Cannot start inference.")
        return

    # Instantiate the model structure (PrithviFloodSegmentationModel must be defined in your notebook)
    final_model = PrithviFloodSegmentationModel(output_classes=1).to(device)
    
    # Load the best trained weights
    final_model.load_state_dict(torch.load(BEST_MODEL_PATH, map_location=device)['model_state_dict'])
    final_model.eval() 
    print("✅ Best model weights loaded successfully. Starting prediction on all chips...")

    # 2. Run Inference on All Chips and Save Masks
    for district, chip_data in all_chips_data.items():
        print(f"\n-> Predicting and stitching for {district}...")
        
        raw_mask_chips_dir = os.path.join(PREDICTION_MASKS_DIR, district)
        os.makedirs(raw_mask_chips_dir, exist_ok=True)
        
        num_chips = min(len(chip_data['pre_flood']), len(chip_data['post_flood']))
        
        for i in range(num_chips):
            pre_tensor = chip_data['pre_flood'][i]
            post_tensor = chip_data['post_flood'][i]
            
            # Create the temporal input (B, C=2, H, W)
            temporal_input = torch.cat([pre_tensor, post_tensor], dim=1).to(device)

            with torch.no_grad():
                # Get model output (logits) and convert to binary mask
                output_logits = final_model(temporal_input) 
                
                # Classification: Apply sigmoid, then round to get 0 or 1
                predicted_mask_tensor = torch.sigmoid(output_logits).round()
                predicted_mask_array = predicted_mask_tensor.squeeze().cpu().numpy().astype(np.uint8)
                
                # --- Get Profile and Save Chip (Conceptual: Requires file path linkage) ---
                # NOTE: You must have a way to link the index 'i' back to the original GeoTIFF 
                # file path to get its metadata profile (transform, CRS, etc.) for saving.
                
                # Placeholder for profile saving (MUST BE IMPLEMENTED IN REAL CODE)
                # This will skip the actual saving, but completes the logic flow.